In [1]:
import os
import sys

from pyspark.sql import SparkSession

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF
from pyspark.ml.feature import StringIndexer


#from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC, OneVsRest

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

import pandas as pd

https://spark.apache.org/docs/3.1.2/

https://spark.apache.org/docs/3.1.2/ml-classification-regression.html#linear-support-vector-machine

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.classification.LinearSVC.html

https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/

https://github.com/tonifuc3m/svm-pyspark/blob/master/svm-spark.ipynb

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.classification.OneVsRest.html#pyspark.ml.classification.OneVsRest



https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html#multiclassclassificationevaluator


https://spark.apache.org/docs/3.1.2/api/java/org/apache/spark/ml/evaluation/MulticlassClassificationEvaluator.html




https://stackoverflow.com/questions/48260412/environment-variables-pyspark-python-and-pyspark-driver-python

In [2]:
os.environ['JAVA_HOME'] = '/opt/jdk'  #Mostra aonde está o JDK
#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "12g") \
    .appName('my-cool-app') \
    .getOrCreate()

sc=spark.sparkContext

21/11/03 15:43:40 WARN Utils: Your hostname, abner-Lenovo-G40-70 resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp2s0)
21/11/03 15:43:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/03 15:43:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
caminho_df_test = 'data_test.csv'
caminho_df_training = 'data_training.csv'
#caminho_df_test = 'menordata_test.csv'
#caminho_df_training = 'menordata_training.csv'

df_test = spark.read.csv(caminho_df_test, header=True, inferSchema=True)
df_training = spark.read.csv(caminho_df_training, header=True, inferSchema=True)

In [6]:
df_test.printSchema()

root
 |-- origem: string (nullable = true)
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



In [7]:
df_training.printSchema()

root
 |-- origem: string (nullable = true)
 |-- topico: string (nullable = true)
 |-- arquivo: integer (nullable = true)
 |-- texto: string (nullable = true)



In [8]:
label_stringIdx = StringIndexer(inputCol = "topico", outputCol = "label", handleInvalid='keep')
tokenization = Tokenizer(inputCol="texto", outputCol="palavras")
remover_stopword = StopWordsRemover(inputCol="palavras", outputCol="palavras_filtradas")
hashingTF = HashingTF(inputCol="palavras_filtradas", outputCol="tf_features")
idf = IDF(inputCol="tf_features", outputCol="tf_idf_features")

In [9]:
lsvc = LinearSVC()
ovr = OneVsRest(featuresCol='tf_idf_features', labelCol='label', classifier=lsvc)

pipelineLSVC = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf, ovr])

In [10]:
lsvcparamGrid = ParamGridBuilder() \
               .addGrid(lsvc.maxIter, [10, 20]) \
               .addGrid(lsvc.regParam, [0.01, 10.0]) \
               .build()     

In [11]:
lsvcCV = CrossValidator(estimator = pipelineLSVC,
                      estimatorParamMaps = lsvcparamGrid,
                      evaluator = MulticlassClassificationEvaluator(predictionCol="prediction"),
                      numFolds = 10)

In [12]:
lsvcCVModel = lsvcCV.fit(df_training)

21/11/03 15:44:15 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:21 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/11/03 15:44:21 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/11/03 15:44:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:24 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:26 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:26 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:27 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/11/03 15:44:

Py4JError: An error occurred while calling o57.evaluate

In [ ]:
df_test_lsvcCVModel = lsvcCVModel.transform(df_test)

In [ ]:
df_predictionAndLabels = df_test_lsvcCVModel.select(['prediction', 'label'])

In [ ]:
labels = df_predictionAndLabels.rdd.map(lambda x: x.label).distinct().collect() # transforma o df para rdd e para poder extrair criar uma lista de labels distintas

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label' )

In [ ]:
vec_fMeasureByLabel = []
qtdClasses = 0
metric_list=[]

for label in sorted(labels):
    precisionByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: label})
    recallByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: label})
    fMeasureByLabel = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: label})
    
    metric_tuple_one = (label, precisionByLabel, recallByLabel,fMeasureByLabel, None , None)
    
    metric_list.append(metric_tuple_one)
    
    vec_fMeasureByLabel.append(fMeasureByLabel)
    qtdClasses +=1 

accuracy = evaluator.evaluate(df_predictionAndLabels, {evaluator.metricName: "accuracy"})
macroF1 = (sum(vec_fMeasureByLabel))/qtdClasses

metric_tuple_two = (None,None,None,None, accuracy, macroF1)

metric_list.append(metric_tuple_two)

In [ ]:
dt_metrics = pd.DataFrame(metric_list)
dt_metrics.columns=(['class', 'precision', 'recall', 'F1', 'accuracy', 'macroF1'])
dt_metrics.to_csv('metrics_svm.csv', index=False)
